In [1]:
from math import ceil
import numpy as np
import pandas as pd
import random

In [2]:
# Importing the data
R_df = pd.read_csv('InputData/R.csv',).drop('user_id\movie_id',axis=1)
R_df.columns = range(len(R_df.columns))
R_df

,0,1,2,3,4,5,6,7,8,9,...,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679
0,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
939,0,0,0,2,0,0,4,5,3,0,...,0,0,0,0,0,0,0,0,0,0
940,5,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Household and Accounts Statistical Model

In [3]:
# Shared Datasets: 

n_one_users = 0
n_two_users = 600
n_three_users = 0
n_four_users = 0

# p_one_user_accounts = 0.60
# p_two_users_accounts = 0.30
# p_three_users_accounts = 0.20
# p_four_users_accounts = 0.10

# p_one_user_accounts = 0.67
# p_two_users_accounts = 0.31
# p_three_users_accounts = 0.016
# p_four_users_accounts = 0.004
# MovieLens 100K
num_users = R_df.shape[0] # total number of users

# Number of users in each account in 100K dataset
# n_one_users = 1+int(num_users*p_one_user_accounts)
# n_two_users = int(num_users*p_two_users_accounts)
# n_three_users = -1+ceil(num_users*p_three_users_accounts)
# n_four_users = ceil(num_users*p_four_users_accounts)

total = n_one_users+n_two_users+n_three_users+n_four_users

n_one_user_accounts = n_one_users
n_two_users_accounts = n_two_users/2
n_three_users_accounts = n_three_users/3
n_four_users_accounts = n_four_users/4

In [4]:
table = pd.DataFrame(
    [[n_one_users, n_two_users, n_three_users, n_four_users, total],
     [n_one_user_accounts,n_two_users_accounts,n_three_users_accounts,n_four_users_accounts,
      n_one_user_accounts+n_two_users_accounts+n_three_users_accounts+n_four_users_accounts]]
    , index=["#users", '#accounts'],columns=[1,2,3,4,'Total']).T.astype(int)

table['#accounts']['Total']

300

In [5]:
table

,#users,#accounts
1,0,0
2,600,300
3,0,0
4,0,0
Total,600,300


In [6]:
number_of_accounts = []
sizes = []
for i in range(4):
    number_of_accounts.append(table['#accounts'].iloc[i])
#     sizes.append(table["#users"].iloc[i])

In [7]:
number_of_accounts

[0, 300, 0, 0]

In [187]:
# Function to merge user rows
def merge_users(user_indices):
    return R_df.iloc[user_indices, :].mean(axis=0)

In [113]:
num_accounts = table['#accounts']['Total']
num_users = R_df.shape[0]
n = R_df.shape[1]

# Allocate users to shared accounts based on the calculated ratios and merge their ratings
np.random.seed(42)  # For reproducibility
user_indices = np.random.permutation(num_users)
account_indices = np.random.permutation(num_accounts)

# Create a new DataFrame for R_sa
R_sa = np.zeros((num_accounts,n),dtype=float)
merge_mapping = []
start_idx = 0
i=0

In [114]:
for size, row in table[:-1].iterrows():
    for _ in range(row['#accounts']):
        selected_users = user_indices[start_idx:start_idx+size]
        R_sa[account_indices[i]] = merge_users(selected_users)
        merge_mapping.append((account_indices[i], list(selected_users))) 
        start_idx += size
        i = i+1

In [115]:
selected_users

array([], dtype=int32)

In [10]:
# Reset the index of the new DataFrame
# R_sa.reset_index(drop=True, inplace=True)
pd.DataFrame(R_sa).to_csv('InputData/R_sa.csv')
# Convert the merge mapping to a DataFrame
merge_mapping_df = pd.DataFrame(merge_mapping,columns=['are mixed in row of R_sa','rows of R'])
merge_mapping_df.to_csv('InputData/SA.csv', index=False)
# Show the shape of the new matrix R_sa and a sample of the data

In [17]:
account_indices

array([524, 313, 382, 582, 455, 456, 175, 340, 232, 457, 594, 337, 309,
       564, 373,  27, 604, 450, 471, 185, 497, 393, 311, 322, 425, 466,
       459, 300,  20,  21, 408, 492, 241, 141, 371, 462, 310, 182, 509,
       488, 154, 596, 104,  67,  57, 426, 417, 271, 316,  51, 424, 427,
       240, 530, 414,  47, 252, 190, 314, 441,  49,  70, 303, 368, 545,
       211, 162,  64, 498, 124, 249, 575, 389,  90, 159,  76, 374, 204,
       487, 247, 357, 275, 411, 123,  29, 244, 439, 168,  65, 381, 569,
       437, 125, 563, 191, 407, 486, 283, 507, 216, 388, 491, 474, 268,
        42, 500,   2, 489,  44,  59, 169, 120,  23, 513, 482, 330, 102,
       554, 387, 430,  98, 468, 115,  93, 473, 469,  92, 237, 220,  71,
       206, 305, 264,  78, 267, 134, 261, 251,  15, 517, 353, 276, 205,
       485, 274, 527, 480, 534, 173, 366, 184, 203, 581,  82,  79,  61,
       166, 479, 342, 231,  73, 383, 361,  43, 420, 347, 280, 432, 101,
       107, 553,  34, 289, 384, 574, 444, 442,  60, 128, 400, 27

In [18]:
merge_mapping

[(524, [96]),
 (313, [265]),
 (382, [810]),
 (582, [23]),
 (455, [30]),
 (456, [280]),
 (175, [568]),
 (340, [259]),
 (232, [331]),
 (457, [323]),
 (594, [422]),
 (337, [467]),
 (309, [286]),
 (564, [893]),
 (373, [868]),
 (27, [638]),
 (604, [538]),
 (450, [499]),
 (471, [481]),
 (185, [334]),
 (497, [848]),
 (393, [770]),
 (311, [925]),
 (322, [39]),
 (425, [363]),
 (466, [764]),
 (459, [307]),
 (300, [444]),
 (20, [713]),
 (21, [704]),
 (408, [76]),
 (492, [817]),
 (241, [595]),
 (141, [371]),
 (371, [165]),
 (462, [755]),
 (310, [250]),
 (182, [882]),
 (509, [436]),
 (488, [239]),
 (154, [107]),
 (596, [67]),
 (104, [174]),
 (67, [158]),
 (57, [139]),
 (426, [754]),
 (417, [731]),
 (271, [306]),
 (316, [532]),
 (51, [694]),
 (424, [802]),
 (427, [63]),
 (240, [235]),
 (530, [765]),
 (414, [899]),
 (47, [86]),
 (252, [692]),
 (190, [723]),
 (314, [778]),
 (441, [516]),
 (49, [660]),
 (70, [587]),
 (303, [326]),
 (368, [218]),
 (545, [209]),
 (211, [656]),
 (162, [70]),
 (64, [928]),

### Generating a dataset that accounts are built with users from different clusters

### Input data

In [8]:
# reading data from clustered users in '00 Weight Matrix Production.ipynb'
%store -r user_indices_of_Type

In [9]:
user_indices_of_Type[3]

[0,
 5,
 9,
 15,
 20,
 21,
 41,
 43,
 55,
 57,
 63,
 69,
 81,
 82,
 86,
 90,
 98,
 101,
 108,
 115,
 116,
 118,
 124,
 143,
 157,
 159,
 176,
 180,
 183,
 187,
 188,
 196,
 206,
 209,
 212,
 214,
 215,
 220,
 232,
 238,
 243,
 245,
 248,
 249,
 253,
 262,
 266,
 274,
 289,
 293,
 297,
 298,
 304,
 306,
 317,
 319,
 324,
 325,
 326,
 331,
 342,
 345,
 346,
 360,
 373,
 378,
 379,
 380,
 384,
 390,
 391,
 393,
 396,
 397,
 408,
 424,
 425,
 441,
 446,
 451,
 452,
 453,
 454,
 455,
 457,
 458,
 465,
 467,
 478,
 483,
 485,
 486,
 487,
 488,
 492,
 497,
 498,
 499,
 513,
 520,
 526,
 532,
 535,
 541,
 542,
 544,
 547,
 552,
 553,
 565,
 566,
 585,
 599,
 607,
 618,
 620,
 621,
 623,
 624,
 626,
 631,
 639,
 642,
 644,
 652,
 658,
 659,
 660,
 662,
 663,
 664,
 665,
 670,
 693,
 696,
 698,
 708,
 709,
 714,
 737,
 747,
 756,
 762,
 765,
 772,
 773,
 775,
 787,
 789,
 794,
 805,
 814,
 822,
 824,
 825,
 829,
 832,
 837,
 839,
 842,
 846,
 847,
 850,
 861,
 866,
 867,
 869,
 870,
 881,
 882,


In [253]:
table

,#users,#accounts
1,0,0
2,600,300
3,0,0
4,0,0
Total,600,300


In [10]:
account_size =[]
number_of_users_in_accounts = []
number_of_accounts = []
for size, row in table[:-1].iterrows():
    account_size.append(size)
    number_of_users_in_accounts.append(row['#users'])
    number_of_accounts.append(row['#accounts'])

In [11]:
number_of_users_in_accounts, number_of_accounts, account_size

([0, 600, 0, 0], [0, 300, 0, 0], [1, 2, 3, 4])

________________________

### Process

In [12]:
np.random.seed(42)  # For reproducibility
user_shuffled_indices_of_Type = []
for i in range(4):
    user_shuffled_indices_of_Type.append(list(np.random.permutation(list(user_indices_of_Type[i]))))

In [13]:
all_accounts = []
new_account = []
random.seed(42)
Types = [0,1,2,3]
i = 0
np.random.seed(42)  # For reproducibility

for i in range(4):
    for account in range(number_of_accounts[i]):
        size = account_size[i]
        Type = random.sample(Types,size)
#         print('account',account,'of size',size, 'availible types',Type)

        new_account = []
        for t in Type:
#             print('from type',t)
            r =np.random.randint(0,high=len(user_shuffled_indices_of_Type[t]))
            new_account.append(user_shuffled_indices_of_Type[t][r])
#             print('newaccount',new_account)

        if len(user_shuffled_indices_of_Type[t]) == 0 :
            Types.remove(t)

#         print(new_account)
#         print('lenT0',len(user_shuffled_indices_of_Type[0]),'lenT1',len(user_shuffled_indices_of_Type[1]),
#               'lenT2',len(user_shuffled_indices_of_Type[2]),'lenT3',len(user_shuffled_indices_of_Type[3]))
#         print('remaining types',Types)
        all_accounts.append(new_account)

In [14]:
all_accounts

[[589, 642],
 [302, 422],
 [273, 247],
 [328, 931],
 [589, 891],
 [326, 16],
 [723, 253],
 [295, 724],
 [819, 386],
 [704, 863],
 [214, 780],
 [331, 863],
 [144, 322],
 [170, 757],
 [663, 490],
 [193, 611],
 [725, 249],
 [210, 620],
 [324, 152],
 [279, 222],
 [144, 819],
 [393, 896],
 [917, 576],
 [348, 221],
 [879, 498],
 [59, 547],
 [323, 605],
 [430, 15],
 [678, 832],
 [260, 390],
 [288, 296],
 [531, 759],
 [746, 934],
 [879, 276],
 [937, 12],
 [879, 325],
 [192, 221],
 [721, 94],
 [169, 569],
 [547, 506],
 [587, 620],
 [796, 89],
 [473, 655],
 [556, 716],
 [270, 269],
 [278, 676],
 [420, 93],
 [895, 427],
 [861, 87],
 [626, 752],
 [150, 194],
 [151, 193],
 [377, 465],
 [214, 177],
 [451, 79],
 [592, 817],
 [82, 431],
 [18, 499],
 [786, 84],
 [540, 456],
 [747, 896],
 [463, 598],
 [380, 471],
 [893, 587],
 [333, 373],
 [833, 406],
 [120, 94],
 [726, 206],
 [294, 807],
 [915, 759],
 [729, 424],
 [770, 17],
 [91, 289],
 [409, 605],
 [264, 150],
 [17, 238],
 [684, 412],
 [404, 104],
 [

### Dataset Generation

In [15]:
num_accounts = table['#accounts']['Total']
num_users = R_df.shape[0]
n = R_df.shape[1]

np.random.seed(42)  # For reproducibility
account_indices = np.random.permutation(num_accounts)

# Create a new DataFrame for R_sa
R_SAD_4C = np.zeros((num_accounts,n),dtype=float)
merge_mapping = []

# Function to merge user rows
def merge_users(user_indices):
    return R_df.iloc[user_indices,:].sum(axis=0)

for i in range(len(all_accounts)):
    R_SAD_4C[account_indices[i]] = merge_users(all_accounts[i])
    merge_mapping.append(all_accounts[i])

In [16]:
merge_mapping

[[589, 642],
 [302, 422],
 [273, 247],
 [328, 931],
 [589, 891],
 [326, 16],
 [723, 253],
 [295, 724],
 [819, 386],
 [704, 863],
 [214, 780],
 [331, 863],
 [144, 322],
 [170, 757],
 [663, 490],
 [193, 611],
 [725, 249],
 [210, 620],
 [324, 152],
 [279, 222],
 [144, 819],
 [393, 896],
 [917, 576],
 [348, 221],
 [879, 498],
 [59, 547],
 [323, 605],
 [430, 15],
 [678, 832],
 [260, 390],
 [288, 296],
 [531, 759],
 [746, 934],
 [879, 276],
 [937, 12],
 [879, 325],
 [192, 221],
 [721, 94],
 [169, 569],
 [547, 506],
 [587, 620],
 [796, 89],
 [473, 655],
 [556, 716],
 [270, 269],
 [278, 676],
 [420, 93],
 [895, 427],
 [861, 87],
 [626, 752],
 [150, 194],
 [151, 193],
 [377, 465],
 [214, 177],
 [451, 79],
 [592, 817],
 [82, 431],
 [18, 499],
 [786, 84],
 [540, 456],
 [747, 896],
 [463, 598],
 [380, 471],
 [893, 587],
 [333, 373],
 [833, 406],
 [120, 94],
 [726, 206],
 [294, 807],
 [915, 759],
 [729, 424],
 [770, 17],
 [91, 289],
 [409, 605],
 [264, 150],
 [17, 238],
 [684, 412],
 [404, 104],
 [

In [44]:
# pd.DataFrame(R_SAD_4C).to_csv('InputData/R_SAD_2U.csv')
# Convert the merge mapping to a DataFrame
# columns=['are mixed in row of R_sa','rows of R']
# merge_mapping_df = pd.DataFrame(np.array(merge_mapping, dtype=object))
merge_mapping_df = pd.DataFrame({'rows of R':merge_mapping})

# merge_mapping_df = pd.DataFrame(np.array(merge_mapping, dtype=object), columns=['rows of R','are mixed in row of R_sa'])
merge_mapping_df.reset_index(inplace=True)
merge_mapping_df.rename(columns={'index':'are mixed in row of R_sa'},inplace=True)
merge_mapping_df.to_csv('InputData/GT_SAD_2U.csv', index=False)

In [45]:
merge_mapping_df

,are mixed in row of R_sa,rows of R
0,0,"[589, 642]"
1,1,"[302, 422]"
2,2,"[273, 247]"
3,3,"[328, 931]"
4,4,"[589, 891]"
...,...,...
295,295,"[343, 108]"
296,296,"[656, 644]"
297,297,"[926, 353]"
298,298,"[181, 733]"
